In [9]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter("ignore")

### First instruction: 
Read in bank_marketing.csv as a pandas DataFrame.

In [10]:
bank_marketing = pd.read_csv('bank_marketing.csv')
bank_marketing.head()

,client_id,age,job,marital,education,credit_default,housing,loan,contact,month,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,0,56,housemaid,married,basic.4y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,1,57,services,married,high.school,unknown,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,2,37,services,married,high.school,no,yes,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,3,40,admin.,married,basic.6y,no,no,no,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,4,56,services,married,high.school,no,no,yes,telephone,may,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [11]:
bank_marketing.dtypes

client_id           int64
age                 int64
job                object
marital            object
education          object
credit_default     object
housing            object
loan               object
contact            object
month              object
day                 int64
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp_var_rate      float64
cons_price_idx    float64
cons_conf_idx     float64
euribor3m         float64
nr_employed       float64
y                  object
dtype: object

### Second intruction: 
Split the data into three DataFrames:

* client
* campaign
* economics

In [12]:
client_col = ['client_id', 'age', 'job', 'marital', 'education','credit_default', 'housing', 'loan']
campaign_col = ['client_id', 'campaign', 'duration', 'pdays', 'previous', 'poutcome', 'y', 'month', 'day']
economics_col = ['client_id', 'emp_var_rate', 'cons_price_idx', 'euribor3m', 'nr_employed']

clients = bank_marketing[client_col]
campaigns = bank_marketing[campaign_col]
economics = bank_marketing[economics_col]

### Third instruction:
Rename the column "client_id" to "id" in client (leave as-is in the other subsets); "duration" to "contact_duration", "previous" to "previous_campaign_contacts", "y" to "campaign_outcome", "poutcome" to "previous_outcome", and "campaign" to "number_contacts" in campaign; and "euribor3m" to "euribor_three_months" and "nr_employed" to "number_employed" in economics.

In [13]:
# In the clients DataFrame
clients.rename(columns={'client_id': 'id'}, inplace=True)

# In campaigns DataFrame
campaigns.rename(columns={ 
                        'duration' : 'contact_duration',
                        'previous' : 'previous_campaign_contacts',
                        'y':'campaign_outcome',
                        'poutcome':'previous_outcome',
                        'campaign':'number_contacts'
                        }, inplace=True)

# In economics DataFrame
economics.rename(columns={
                        'euribor3m':'euribor_three_months',
                        'nr_employed':'number_employed'
                        }, inplace=True)

### Fourth instruction:
Clean the "education" column, changing "." to "_" and "unknown" to NumPy's null values.

In [14]:
clients['education'] = clients['education'].str.replace('\.','_')
clients['education'] = clients['education'].replace('unknown',np.nan, inplace=True)

In [15]:
# Check for . and 'unknown'
to_check=['.', 'unknown']
counter_dot = clients['education'].str.contains(to_check[0]).sum()
counter_unknown = (clients['education'] == to_check[1]).sum()

if counter_dot == 0 and counter_unknown == 0:
    print("Education's column is clean.")
else:
    print("Education's column has still {} dot(s). and {} unknown(s) values".format(counter_dot, counter_unknown))

Education's column is clean.


### Fifth instruction:
Remove periods from the "job" column.

In [16]:
clients['job'] = clients['job'].str.replace('.', '')

# To find a "." and stop ASAP 
if '.' not in clients['job'].values:
    print("There are no dots in the job's column.")
else: 
    print("There are still dots in the job's column.")

There are no dots in the job's column.


### sixth instruction:
Convert "success" and "failure" in the "previous_outcome" and "campaign_outcome" columns to binary (1 or 0), along with the changing "nonexistent" to NumPy's null values in "previous_outcome".

In [17]:
campaigns['previous_outcome'] = campaigns['previous_outcome'].map({'success':1, 'failure':0})
campaigns['campaign_outcome'] = campaigns['campaign_outcome'].map({'success':1, 'failure':0})

campaigns['previous_outcome'] = campaigns['previous_outcome'].replace('nonexistent', np.nan, inplace=True)

if ("success" or "failure") not in campaigns['previous_outcome'].values and\
    ("success" or "failure") not in campaigns['campaign_outcome'].values and\
    ("nonexistent") not in campaigns['previous_outcome'].values:
    print("Columns were converted and changed successfully!")
else:
    print("Columns were not converted successfully!")

Columns were converted and changed successfully!


### Seventh instruction:
Add a column called campaign_id in campaign, where all rows have a value of 1.

In [18]:
campaigns['campaign_id'] = 1

campaigns.head(3)

,client_id,number_contacts,contact_duration,pdays,previous_campaign_contacts,previous_outcome,campaign_outcome,month,day,campaign_id
0,0,1,261,999,0,None,NaN,may,13,1
1,1,1,149,999,0,None,NaN,may,19,1
2,2,1,226,999,0,None,NaN,may,23,1


### Eighth instruction:
Create a datetime column called last_contact_date, in the format of "year-month-day", where the year is 2022, and the month and day values are taken from the "month" and "day" columns.

In [19]:
# We fix the year to 2022
campaigns['year'] = 2022

# to create the last_contact_date colum
campaigns['last_contact_date'] = pd.to_datetime(
                                campaigns['year'].astype(str) + '-' +
                                campaigns['month'].astype(str) + '-' +
                                campaigns['day'].astype(str),
                                errors = 'coerce')
# Check if column last_contact_date was created

if 'last_contact_date' in campaigns.columns:
    print('The column was created!')
else:
    print('The column was not created!')

The column was created!


### Ninth instruction:
Remove any redundant data that might have been used to create new columns, ensuring the columns in each subset of the data match the table displayed in the notebook.

In [20]:
campaigns = campaigns.drop(['year', 'month', 'day'], axis=1)

print("Clients\n{}\n\nCampaigns\n{}\n\nEconomics\n{}\n".format(clients.head(), campaigns.head(), economics.head()))


Clients
   id  age        job  marital education credit_default housing loan
0   0   56  housemaid  married      None             no      no   no
1   1   57   services  married      None        unknown      no   no
2   2   37   services  married      None             no     yes   no
3   3   40      admin  married      None             no      no   no
4   4   56   services  married      None             no      no  yes

Campaigns
   client_id  number_contacts  contact_duration  pdays  \
0          0                1               261    999   
1          1                1               149    999   
2          2                1               226    999   
3          3                1               151    999   
4          4                1               307    999   

   previous_campaign_contacts previous_outcome  campaign_outcome  campaign_id  \
0                           0             None               NaN            1   
1                           0             None          

### Tenth instruction:
Save the three DataFrames to csv files without an index as client.csv, campaign.csv, and economics.csv respectively.

In [21]:
clients.to_csv('client.csv', index=False)
clients.to_csv('campaign.csv', index=False)
clients.to_csv('economics.csv', index=False)


### Eleventh instruction:
Create a Python variable called client_table, containing SQL code as a multi-line string to create a table called client using values from client.csv.

In [22]:
client_table = '''
                CREATE TABLE client (
                id SERIAL PRIMARY KEY,
                age INT,
                job TEXT,
                marital TEXT,
                education TEXT,
                credit_default BOOLEAN,
                housing BOOLEAN,
                loan BOOLEAN
                );
-- To SQL table from CSV file
\copy client FROM 'client.csv' DELIMITER ',' CSV HEADER;
                '''

### Twelfth instruction:
Create a Python variable called campaign_table, containing SQL code as a multi-line string to create a table called campaign using values from campaign.csv.

In [23]:
#print(campaigns.head())

campaign_table = '''
                CREATE TABLE campaign (
                campaign_id SERIAL PRIMARY KEY,
                client_id SERIAL REFERENCES client(id),
                number_contacts INT,
                contact_duration INT,
                pdays INT,
                previous_campaign_contacts INT,
                previous_outcome BOOLEAN,
                campaign_outcome BOOLEAN,
                last_contact_date DATE
                );
-- To SQL table from CSV file
\copy campaign FROM 'campaign.csv' DELIMITER ',' CSV HEADER;
'''

### Thirteenth instruction:
Create a Python variable called economics_table, containing SQL code as a multi-line string to create a table called economics using values from economics.csv.

In [ ]:
#print(economics.head())

economics_table = '''
                CREATE TABLE economics (
                client_id SERIAL REFERENCES client (id),
                emp_var_rate FLOAT,
                cons_price_idx FLOAT,
                euribor_three_months FLOAT,
                number_employed FLOAT);
-- To SQL table from CSV file
\copy economics FROM 'economics.csv' DELIMITER ',' CSV HEADER;
'''